In [82]:
# process downloaded sen files
from pyroSAR.snap import geocode
import os


files = [os.path.join('data/download/', file) for each in os.listdir('data/download/') if each.endswith('.zip')]
# satfiles = [os.path.join(root, name)
#             for root, dirs, files in os.walk('../sen1ost/data/download/SAR/GRD/2021')
#             for name in files
#             if name.endswith("zip")]
# satfiles = satfiles[0:3]
site_shp = 'data/aoi/aoi_tmp/aoi_tmp.shp'

for single_file in files:
# for single_file in satfiles:
    geocode(infile=single_file,
        outdir='data/out',
        spacing=10,
        shapefile=site_shp,
        polarizations=['VV', 'VH'],
        scaling='dB',
        cleanup=True,
        tmpdir='data/tmp',
        groupsize=8)



In [ ]:
# cut to aio size
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os

tif_files = [file for file in os.listdir('data/out/raw') if file.endswith(".tif")]
site_shp = 'data/aoi/flotow.shp'

# Open the .tif file
for file in tif_files:
    with rasterio.open(os.path.join('data/out/raw', file)) as src:
        # Open the .shp file using geopandas
        shapefile = gpd.read_file(site_shp)

        # Clip the .tif file using the .shp file geometry
        clipped, transform = mask(src, shapefile.geometry, crop=True)
        out_profile = src.profile.copy()
        out_profile.update({'width': clipped.shape[2],'height': clipped.shape[1], 'transform': transform})

        # Save the clipped .tif file
        with rasterio.open(os.path.join('data/out/cut', file), 'w', **out_profile) as dst:
            dst.write(clipped)
